In [2]:
import scanpy as sc
import anndata as an
import pandas as pd

In [12]:
fpath = "/scratch/indikar_root/indikar1/shared_data/geneformer/resources/token_mapping.csv"

df = pd.read_csv(fpath)
df.head()

,gene_id,token_id,gene_name,nonzero_median,gene_version,gene_biotype,Chromosome,Start,End,scenic_tf
0,<pad>,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,<mask>,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,ENSG00000000003,2,TSPAN6,2.001186,15.0,protein_coding,X,100627107.0,100639991.0,False
3,ENSG00000000005,3,TNMD,3.228213,6.0,protein_coding,X,100584935.0,100599885.0,False
4,ENSG00000000419,4,DPM1,2.218874,14.0,protein_coding,20,50934866.0,50959140.0,False


In [11]:
adata.obs['STD.CellType'].value_counts()

STD.CellType
Refined.HSC    3703
MEP            3622
CD4            2544
MPP            2302
EryP           2248
ProB           2104
CD8            1963
MKP            1932
Mono           1434
HSC            1293
MDP            1138
NK             1017
CLP             947
GMP             933
B               748
LMPP            459
CMP             416
Plasma           63
cDC              58
pDC              24
unknown           0
Name: count, dtype: int64

In [ ]:
break

In [9]:
adata.obs['dataset'].value_counts()

dataset
LinNegCD34PosCD164Pos     6343
LinNegCD34NegCD164high    4434
LinNegCD34lowCD164high    4266
CMP                       1576
HSC                       1282
MEP                       1211
GMP                       1012
PreBNK                     592
LinNegCD34NegCD164low      358
MPP                        215
MLP                        123
Name: count, dtype: int64

In [ ]:
def aggregate_gene_counts(adata):
    """
    Aggregates raw counts from an AnnData object at the gene level.

    Args:
        adata: An AnnData object containing gene expression data in a layer named 'raw_counts'.

    Returns:
        pandas.DataFrame: A DataFrame with gene names as index and summed raw counts as columns.
    """
    # Extract raw counts and transpose for gene-wise analysis
    df = adata.to_df(layer='raw_counts').T
    
    # Reset index to a column named 'gene_name'
    df.reset_index(names='gene_name', inplace=True)

    # Pre-aggregation info for debugging
    print(f"Initial shape: {df.shape}")

    # Extract base gene names (remove potential suffixes)
    df['gene_name'] = df['gene_name'].astype(str).str.split(".", n=1, expand=True)[0]  

    # Group by gene name and sum counts across all cells
    df = df.groupby('gene_name').sum()

    # Post-aggregation info
    print(f"Final shape: {df.shape}")

    return df.T

df = aggregate_gene_counts(adata)
df.head()

In [ ]:
df = adata.to_df(layer='raw_counts').T
df = df.reset_index(names='gene_name')
print(f"{df.shape=}")
df['gene_name'] = df['gene_name'].apply(lambda x: x.split(".")[0])
df = df.groupby('gene_name').sum()
print(f"{df.shape=}")
df.head()

In [ ]:
def create_anndata_from_dataframe(df, adata):
    """
    Creates an AnnData object from a pandas DataFrame and incorporates metadata from an existing AnnData object.

    Args:
        df: A pandas DataFrame where rows are features (e.g., genes) and columns are samples.
        adata: An existing AnnData object containing metadata to be transferred (obs and var_names).

    Returns:
        anndata.AnnData: The newly created AnnData object.
    """
    # Create AnnData from the DataFrame's numerical values
    pdf = an.AnnData(df.to_numpy())

    # Copy observation metadata (e.g., cell IDs, annotations)
    pdf.obs = adata.obs.copy()

    # Create variable annotation with Ensembl IDs
    pdf.var = pd.DataFrame({'ensembl_id': df.columns})

    # Set variable and observation names from the DataFrame
    pdf.var_names = df.columns
    pdf.obs_names = df.index

    return pdf

pdf = create_anndata_from_dataframe(df, adata)
pdf

In [ ]:
pdf.obs.head()

In [ ]:
pdf = an.AnnData(df.to_numpy())
pdf.obs = adata.obs.copy()
pdf.var = pd.DataFrame({'ensembl_id' : df.index})
pdf.var_names = df.index
pdf.obs_names = df.columns

pdf


In [ ]:
pdf.var.head()

In [ ]:
adata.var['']

In [ ]:
fpath = "/nfs/turbo/umms-indikar/shared/projects/geneformer/data/rajapakse_lab_data_jpic.h5ad"

adata = sc.read_h5ad(fpath)
adata

In [ ]:
adata.obs

In [ ]:
fpath = "/scratch/indikar_root/indikar1/cstansbu/HSC/scanpy/clustered.anndata.h5ad"

adata = sc.read_h5ad(fpath)
adata

In [ ]:
adata.obs['celltype'].value_counts()

In [ ]:
adata.obs[adata.obs['celltype'] == 'FB']['organ_tissue'].value_counts()